# Purpose
The purpose of this notebook is to replicate notebook `_09-tb-Calculate-lognormal-MIXL-probs` using the pytorch implementation of MIXLB.

In [1]:
import sys
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.sparse as sparse
import pandas as pd

import pylogit as pl
import pylogit.mixed_logit_calcs as mlc
import pylogit.choice_tools as ct

sys.path.insert(0, '../../')
import src.models.mixlb as mixlb

# Load needed data

In [2]:
car_df = pd.read_csv("../../data/processed/model_ready_car_data.csv")
forecast_df = pd.read_csv("../../data/processed/forecast_car_data.csv")

# Estimate the MNL model

In [3]:
# Create specification and name dictionaries
mnl_spec, mnl_names = OrderedDict(), OrderedDict()

orig_cols_and_display_names =\
    [("neg_price_over_log_income", 'Neg Price over log(income)'),
     ('range_over_100', 'Range (units: 100mi)'),
     ("neg_acceleration_over_10", 'Neg Acceleration (units: 0.1sec)'),
     ('top_speed_over_100', 'Neg Top speed (units: 0.01mph)'),
     ("neg_pollution", 'Neg Pollution'),
     ('vehicle_size_over_10', 'Size'),
     ('big_enough', 'Big enough'),
     ('luggage_space', 'Luggage space'),
     ("neg_tens_of_cents_per_mile", 'Neg Operation cost'),
     ('station_availability', 'Station availability'),
     ('sports_utility_vehicle', 'Sports utility vehicle'),
     ('sports_car', 'Sports car'),
     ('station_wagon', 'Station wagon'),
     ('truck', 'Truck'),
     ('van', 'Van'),
     ('electric', 'EV'),
     ('electric_commute_lte_5mi', 'Commute < 5 & EV'),
     ('electric_and_college', 'College & EV'),
     ('compressed_natural_gas', 'CNG'),
     ('methanol', 'Methanol'),
     ('methanol_and_college', 'College & Methanol'),
     ('non_ev', 'Non Electric-Vehicle'),
     ('non_cng', 'Non Compressed Natural Gas')]

for col, display_name in orig_cols_and_display_names:
    mnl_spec[col] = 'all_same'
    mnl_names[col] = display_name

In [4]:
# Estimate an mnl with the same basic specification as the MIXL.
mnl_model =\
    pl.create_choice_model(data=car_df,
                           alt_id_col='alt_id',
                           obs_id_col='obs_id',
                           choice_col='choice',
                           specification=mnl_spec,
                           model_type='MNL',
                           names=mnl_names)

mnl_model.fit_mle(np.zeros(len(mnl_names)),
                  constrained_pos=[-2, -1])

# Look at the estimation results
mnl_model.get_statsmodels_summary()

Log-likelihood at zero: -8,338.8486
Initial Log-likelihood: -8,338.8486


/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/pylogit/choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 0.20 seconds.
Final log-likelihood: -7,391.8300


/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/pylogit/base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/timothyb0912/minimamba/envs/checkYourself/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                4,654
Model:             Multinomial Logit Model   Df Residuals:                    4,631
Method:                                MLE   Df Model:                           23
Date:                     Wed, 10 Jun 2020   Pseudo R-squ.:                   0.114
Time:                             11:16:54   Pseudo R-bar-squ.:               0.111
AIC:                            14,829.660   Log-Likelihood:             -7,391.830
BIC:                            14,977.906   LL-Null:                    -8,338.849
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Neg Price over log(income)           0.1854      0.027      6.796      0.000       0.132       0.239
Range (units: 100mi)                 0.3501      0.027     13.052      0.000       0.298       0.403
Neg Acceleration (units: 0.1sec)     0.7160      0.111      6.472      0.000       0.499       0.933
Neg Top speed (units: 0.01mph)       0.2612      0.081      3.228      0.001       0.103       0.420
Neg Pollution                        0.4441      0.102      4.367      0.000       0.245       0.643
Size                                 0.9345      0.316      2.953      0.003       0.314       1.555
Big enough                           0.1432      0.077      1.853      0.064      -0.008       0.295
Luggage space                        0.5009      0.191      2.623      0.009       0.127       0.875
Neg Operation cost                   0.7679      0.076     10.131      0.000       0.619       0.916
Station availability                 0.4133      0.096      4.294      0.000       0.225       0.602
Sports utility vehicle               0.8201      0.141      5.830      0.000       0.544       1.096
Sports car                           0.6370      0.148      4.298      0.000       0.347       0.928
Station wagon                       -1.4367      0.062    -23.139      0.000      -1.558      -1.315
Truck                               -1.0168      0.049    -20.753      0.000      -1.113      -0.921
Van                                 -0.7989      0.047    -16.864      0.000      -0.892      -0.706
EV                                  -0.1786        nan        nan        nan         nan         nan
Commute < 5 & EV                     0.1983      0.084      2.374      0.018       0.035       0.362
College & EV                         0.4426      0.109      4.058      0.000       0.229       0.656
CNG                                  0.3450        nan        nan        nan         nan         nan
Methanol                             0.3134      0.103      3.051      0.002       0.112       0.515
College & Methanol                   0.2284      0.089      2.576      0.010       0.055       0.402
Non Electric-Vehicle                      0        nan        nan        nan         nan         nan
Non Compressed Natural Gas                0        nan        nan        nan         nan         nan
====================================================================================================
"""

In [5]:
# Create a model object mnl with the same basic specification as the MIXL.
# To be used later to extract the design matrix
forecast_model =\
    pl.create_choice_model(data=forecast_df,
                           alt_id_col='alt_id',
                           obs_id_col='obs_id',
                           choice_col='choice',
                           specification=mnl_spec,
                           model_type='MNL',
                           names=mnl_names)

# Initialize the MIXL model

In [6]:
# Instantiate the model
mixl_model = mixlb.MIXLB()

# Set the model parameters to the final estimates from Brownstone & Train (1998),
# taking care of the typo from the published paper.
mean_array =\
    np.array([-1.5983748481622846, #-5.999,
              -0.877,
              -0.302,
              -1.364,
              -0.711,
               1.541,
              -1.748,
               1.563,
              -0.071,
              -0.741,
               0.897,
               0.698,
              -1.508,
              -1.094,
              -0.819,
              -0.905,
               0.359,
               0.770,
               0.621,
               0.476,
               0.335,
               0,
               0])

std_dev_array =\
    np.array([6.808, 5.380, 2.289, 0.971])

paper_estimates_array =\
    np.concatenate((mean_array, std_dev_array), axis=0)
mixl_model.set_params_numpy(paper_estimates_array)

# Create arguments to calculate model probabilities

In [7]:
def create_sparse_mapping_torch(id_array):
    mapping_scipy =\
        ct.create_sparse_mapping(id_array).tocoo()
    torch_mapping_indices =\
        torch.LongTensor(torch.from_numpy(
            np.concatenate((mapping_scipy.row[None, :],
                            mapping_scipy.col[None, :]),
                           axis=0).astype(np.int_)))
    torch_mapping_values =\
        (torch.from_numpy(mapping_scipy.data.astype(np.float32))
              .double())
    num_rows = mapping_scipy.data.size
    num_cols = ct.get_original_order_unique_ids(id_array).size
    mapping_torch =\
        sparse.FloatTensor(
            torch_mapping_indices,
            torch_mapping_values,
            torch.Size([num_rows, num_cols]))
    return mapping_torch

In [8]:
# Get the design matrix from the original and forecast data
orig_design_matrix_np = mnl_model.design
orig_design_matrix =\
    torch.tensor(orig_design_matrix_np.astype(np.float32))


forecast_design_np = forecast_model.design
forecast_design_matrix =\
    torch.tensor(forecast_design_np.astype(np.float32))

# Get the rows_to_obs and rows_to_mixers matrices.
rows_to_obs =\
    create_sparse_mapping_torch(car_df[mnl_model.obs_id_col].values)
rows_to_mixers =\
    create_sparse_mapping_torch(car_df[mnl_model.obs_id_col].values)



####
# Get the normal random variates.
####
# Determine the number of draws being used for the mixed logit
num_draws = 250
# Determine the number of observations with randomly distributed
# sensitivities
num_mixers = car_df.obs_id.unique().size

# Get the random draws needed for the draws of each coeffcient
# Each element in the list will be a 2D ndarray of shape
# num_mixers by num_draws
normal_rvs_list_np =\
    mlc.get_normal_draws(num_mixers,
                         num_draws,
                         mixl_model.design_info.num_mixing_vars,
                         seed=601)
normal_rvs_list = [torch.from_numpy(x).double() for x in normal_rvs_list_np]

# Compare MIXL probabilities to MNL

In [9]:
# Compute the MIXL probabilities
mixl_probs =\
    mixl_model.forward(design_2d=orig_design_matrix,
                       rows_to_obs=rows_to_obs,
                       rows_to_mixers=rows_to_mixers,
                       normal_rvs_list=normal_rvs_list)
# Compute the MIXL log-likelihood
torch_choices =\
    torch.from_numpy(mnl_model.choices.astype(np.float32)).double()
mixl_log_likelihood =\
    torch.sum(torch_choices * torch.log(mixl_probs))
mixl_log_likelihood

tensor(-7370.0864, dtype=torch.float64, grad_fn=<SumBackward0>)

In [10]:
# Compare the MIXL to MNL log-likelihoods
msg = 'MIXL: {:,.2f}\nMNL:  {:,.2f}'
print(msg.format(mixl_log_likelihood.item(), mnl_model.llf))

MIXL: -7,370.09
MNL:  -7,391.83


In [11]:
# Compute the gradients
mixl_log_likelihood.backward()
# Extract the gradients
paper_gradients = mixl_model.get_grad_numpy()
print('Gradients:\n{}'.format(paper_gradients))
# Zero the gradients as if we were performing optimization
mixl_model.zero_grad()
# Extract the probabilities as an array
mixl_probs_array = mixl_probs.detach().numpy()
# Make sure none of the predicted probabilities are NaN
msg = '\nAny NaN probabilities? {}\n'
print(msg.format(np.isnan(mixl_probs_array).any()))
# Describe the computed probabilities
print(pd.Series(mixl_probs_array).describe())

Gradients:
[  0.52739913  16.80788119  -2.03877212  -1.5663477   -4.6216078
   0.34231704   0.18036642   3.63172369  -2.92595559   4.3653854
   1.12133433  -0.70962555   0.82817194  -0.70474704   2.46969284
   0.677335     0.69274961   0.4105395  -34.60967672  29.33451904
  22.59093556  -0.677335    34.60967672   0.10889218  -1.29851708
   0.89968333  -4.38320548]

Any NaN probabilities? False

count    27924.000000
mean         0.166667
std          0.111085
min          0.008662
25%          0.080818
50%          0.135765
75%          0.228371
max          0.646645
dtype: float64


In [12]:
# Compute the forecast probabilities
forecast_probs =\
    mixl_model.forward(design_2d=forecast_design_matrix,
                       rows_to_obs=rows_to_obs,
                       rows_to_mixers=rows_to_mixers,
                       normal_rvs_list=normal_rvs_list)
# Zero the gradients as if we were performing optimization
mixl_model.zero_grad()
# Extract the probabilities as an array
forecast_probs_array = forecast_probs.detach().numpy()

In [13]:
# Ensure the forecast probabilities for large gas cars are
# higher than the original probabilities for large gas cars
large_gas_car_idx = ((car_df['body_type'] == 'regcar') &
                     (car_df['vehicle_size'] == 3) &
                     (car_df['fuel_type'] == 'gasoline')).values
num_stupid_forecasts =\
    ((forecast_probs_array > mixl_probs_array)[large_gas_car_idx]).sum()
print("{:,} stupid forecasts".format(num_stupid_forecasts))

# Look at the mixed logits predicted change in the market share
# of large gas cars after the price increase.
large_gas_car_share_change =\
    (((forecast_probs_array[large_gas_car_idx]).sum() -
     (mixl_probs_array[large_gas_car_idx]).sum()) /
     (mixl_probs_array[large_gas_car_idx]).sum())
msg = 'The predicted change in market share of large gas cars is {:.3%}'
print(msg.format(large_gas_car_share_change))

0 stupid forecasts
The predicted change in market share of large gas cars is -12.205%


# Findings
The most unexpected finding was that none of the parameter gradients was near zero.

This implies that the parameters that maximize the simulated log-likelihood function may be far from the current parameters.

Who knows if the true maximum likelihood parameters will result in a predictive model with the same qualitative findings.